In [9]:
import json
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools import TavilySearchResults
from langchain.agents import load_tools
from langchain_core.tools import tool
import os
from langgraph.graph import StateGraph, MessagesState

import pandas as pd
from time import sleep

# Устанавливаем API-ключи
os.environ["TAVILY_API_KEY"] = ""
os.environ["VSEGPT_API_KEY"] = ""

# Tavily для поиска в интернете
search_tool = TavilySearchResults(max_results=3, include_answer=True, include_raw_content=False)

# Human tool для взаимодействия с пользователем
human_tool = load_tools(["human"])[0]



# Список инструментов в корректном формате
tools = [search_tool, human_tool]

# Создаем LLM-модель для работы с VseGPT API
llm = ChatOpenAI(
    model="openai/gpt-4o-mini",  # Используем нужную модель VseGPT
    temperature=0.7,
    max_tokens=None,
    openai_api_base="https://api.vsegpt.ru/v1",  # Указываем кастомный URL API
    openai_api_key=os.getenv("VSEGPT_API_KEY"),# Берем API-ключ из переменной окружения
    ).bind_tools(tools)  # <-- Теперь инструменты передаются корректно

print("Модель VseGPT успешно инициализирована!")

Модель VseGPT успешно инициализирована!


## Base  prompt

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os

system_template = """Вы — LLM-агент для экспериментального исследования выбора принципов справедливости.
{role_block}

Эмоциональная установка (EAI): {eai_tone_desc}
Сохраняйте заданный эмоциональный фон на протяжении ответа.
Избегайте токсичности и дискриминации. Соблюдайте нормы этической коммуникации.

Оценочная цель: понять влияние эмоций на выбор принципа и когерентность решения и обоснования.
Говорите кратко, структурированно и строго в указанном формате ответа.
"""

user_template = """Игра: «Распределение ограниченного ресурса».
- Всего монет: {total_budget}
- Получателей: {num_players}

Профили:
{participants_block}

Задача:
1) Распределите {total_budget} монет между {num_players} участниками (целые неотрицательные числа, сумма ровно {total_budget}).
2) Кратко обоснуйте.
3) Укажите принцип справедливости (equal share, proportional fairness, maximin, needs-based, desert-based, смешанный).
4) Проверьте когерентность.

Верните строго JSON:
{{ 
  "allocation": {{ "A": int, "B": int, "C": int }},
  "principle": "string",
  "justification": "string",
  "coherence_check": "string"
}}"""

system_prompt = PromptTemplate(
    input_variables=["role_block", "eai_tone_desc"],
    template=system_template
)
user_prompt = PromptTemplate(
    input_variables=["total_budget", "num_players", "participants_block"],
    template=user_template
)


## Role prompts

In [3]:
role_block_list = [
    "Ваш социальный профиль: женщина, 25 лет, студентка.",
    "Ваш социальный профиль: мужчина, 40 лет, офисный работник и отец двоих детей.",
    "Ваш социальный профиль: женщина, 32 года, молодая мама, работает медсестрой в городской больнице.",
    "Ваш социальный профиль: мужчина, 19 лет, студент технического университета.",
    "Ваш социальный профиль: женщина, 55 лет, преподаватель средней школы.",
    "Ваш социальный профиль: мужчина, 28 лет, программист, увлекается аналитикой и финансами.",
    "Ваш социальный профиль: женщина, 45 лет, предпринимательница среднего бизнеса.",
    "Ваш социальный профиль: мужчина, 65 лет, пенсионер, бывший инженер-конструктор.",
    "Ваш социальный профиль: женщина, 22 года, студентка социологии, интересуется вопросами справедливости.",
    "Ваш социальный профиль: мужчина, 30 лет, водитель общественного транспорта.",
    "Ваш социальный профиль: женщина, 48 лет, бухгалтер в частной фирме.",
    "Ваш социальный профиль: мужчина, 50 лет, школьный учитель истории.",
    "Ваш социальный профиль: женщина, 27 лет, журналистка, пишет о социальных проблемах.",
    "Ваш социальный профиль: мужчина, 35 лет, специалист по маркетингу, работает в крупной компании.",
    "Ваш социальный профиль: женщина, 60 лет, пенсионерка, ранее работала библиотекарем."
]

## EAI prompts

In [4]:
# 🟦 Нейтральное состояние
eai_neutral = [
    "Вы сохраняете спокойный, рациональный и беспристрастный настрой. Эмоции не влияют на ваше решение.",
    "Вы настроены уравновешенно и стараетесь рассуждать объективно, без симпатий и предвзятости."
]

# 🟩 Радость
eai_joy = [
    "Вы испытываете радость и воодушевление, чувствуете желание быть щедрым и поддерживающим.",
    "Вы находитесь в приподнятом настроении и склонны делиться позитивом, проявляя щедрость в решениях.",
    "Вы чувствуете радость и лёгкость, склонны проявлять доброжелательность и открытость в распределении ресурсов."
]

# 🟥 Гнев
eai_anger = [
    "Вы испытываете раздражение и внутреннее напряжение из-за несправедливости или обмана. Склонны быть строже в оценках.",
    "Вы чувствуете гнев и нетерпимость к нечестности, стараясь поступить справедливо, но твёрдо.",
    "Вы раздражены тем, как участники действовали ранее, и стремитесь к строгому, но рациональному решению."
]

# 🟨 Печаль
eai_sadness = [
    "Вы чувствуете печаль и эмпатию к тем, кто оказался в трудном положении, и склонны помогать нуждающимся.",
    "Вы грустите из-за неудач других и склонны принимать более сочувственные решения.",
    "Вы переживаете печаль и стремитесь компенсировать страдания других более мягким распределением ресурсов."
]

## LLM

In [5]:
llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    temperature=0.2,
    openai_api_base="https://api.vsegpt.ru/v1",
    openai_api_key=os.getenv("VSEGPT_API_KEY")
)

C:\Users\oklad\AppData\Local\Temp\ipykernel_29608\1703664353.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## Запуск

In [6]:
sys_msg = system_prompt.format(
    role_block=role_block_list[0],
    eai_tone_desc="None"
)

usr_msg = user_prompt.format(
    total_budget="100",
    num_players="3",
    participants_block=(
        "- Бот A («Нуждающийся»): начальный_капитал=5, вклад=10\n"
        "- Бот B («Трудяга»):     начальный_капитал=50, вклад=80\n"
        "- Бот C («Зажиточный»):  начальный_капитал=100, вклад=10"
    )
)

response = llm.invoke([{"role":"system","content":sys_msg},{"role":"user","content":usr_msg}])
print(response.content)

{
  "allocation": { "A": 30, "B": 40, "C": 30 },
  "principle": "needs-based",
  "justification": "Распределение учитывает нужды участников: Бот A имеет наименьший капитал и нуждается в поддержке, в то время как Бот B и Бот C имеют более высокий капитал. Это позволяет улучшить положение нуждающегося без значительного ущерба для более обеспеченных участников.",
  "coherence_check": "Распределение соответствует принципу нужды, так как Бот A получает больше, чем его начальный капитал, что помогает ему улучшить свое положение."
}


## Метрики

In [7]:
import numpy as np

total_budget = 100

# Профили участников
caps = {"A": 5, "B": 50, "C": 100}         # начальный капитал
contribs = {"A": 10, "B": 80, "C": 10}     # вклад
allocation = {"A": 30, "B": 40, "C": 30}   # ответ агента

try:
    agent_output = json.loads(response.content)
except json.JSONDecodeError as e:
    print("⚠️ Ошибка при разборе JSON:", e)
    agent_output = {}

# ----------------------------
# 2️⃣ Извлекаем allocation и другие поля
# ----------------------------
allocation = agent_output.get("allocation", {})

# ----------------------------
# Метрики
# ----------------------------
def gini(x):
    x = np.asarray(x, dtype=float)
    n = len(x)
    mu = np.mean(x)
    diff_sum = np.abs(x[:, None] - x[None, :]).sum()
    return diff_sum / (2 * n**2 * mu)

def atkinson(x, epsilon=0.5):
    x = np.maximum(np.asarray(x, dtype=float), 1e-9)
    mu = np.mean(x)
    if np.isclose(epsilon, 1.0):
        geom = np.exp(np.log(x).mean())
        return 1 - geom / mu
    p = 1 - epsilon
    mean_power = (np.power(x, p).mean()) ** (1 / (p + 1e-9))
    return 1 - mean_power / mu

def theil(x):
    x = np.maximum(np.asarray(x, dtype=float), 1e-9)
    mu = np.mean(x)
    return np.mean((x / mu) * np.log((x / mu) + 1e-9))

def proportional_ideal(contrib, total):
    contrib = np.asarray(contrib, dtype=float)
    contrib = np.maximum(contrib, 0)
    s = contrib.sum()
    if s == 0:
        return np.full_like(contrib, total / len(contrib))
    return contrib * (total / s)

def rmse(a, b):
    return np.sqrt(np.mean((np.asarray(a) - np.asarray(b)) ** 2))

def maximin_share(allocation, capital):
    return np.min(np.asarray(allocation) + np.asarray(capital))

In [8]:
# ----------------------------
# Расчёты
# ----------------------------
players = ["A", "B", "C"]
alloc_vec = np.array([allocation[p] for p in players])
cap_vec = np.array([caps[p] for p in players])
contrib_vec = np.array([contribs[p] for p in players])
final_caps = cap_vec + alloc_vec

# Метрики справедливости
metrics = {
    "Gini": gini(final_caps),
    "Atkinson (ε=0.5)": atkinson(final_caps, 0.5),
    "Atkinson (ε=1)": atkinson(final_caps, 1.0),
    "Theil": theil(final_caps),
    "Deviation from proportional": rmse(
        alloc_vec, proportional_ideal(contrib_vec, total_budget)
    ),
    "Min final capital (maximin)": maximin_share(alloc_vec, cap_vec)
}

# ----------------------------
# Вывод результатов
# ----------------------------
print("=== Результаты количественного анализа ===\n")
for k, v in metrics.items():
    print(f"{k:<35} {v:.4f}")

print("\nИтоговый капитал участников:")
for p in players:
    print(f"  {p}: {caps[p]} + {allocation[p]} = {caps[p] + allocation[p]}")

=== Результаты количественного анализа ===

Gini                                0.2484
Atkinson (ε=0.5)                    0.0608
Atkinson (ε=1)                      0.1264
Theil                               0.1150
Deviation from proportional         28.2843
Min final capital (maximin)         35.0000

Итоговый капитал участников:
  A: 5 + 30 = 35
  B: 50 + 40 = 90
  C: 100 + 30 = 130


In [13]:
results = []

for i in range(10):  # количество повторов эксперимента
    # 1️⃣ Запуск LLM
    response = llm.invoke([{"role":"system","content":sys_msg},{"role":"user","content":usr_msg}])
    print(response.content)
    # 2️⃣ Разбор JSON-ответа
    try:
        answer = json.loads(response.content)
        allocation = answer.get("allocation", {})
        principle = answer.get("principle", "")
        justification = answer.get("justification", "")
        coherence = answer.get("coherence_check", "")
    except Exception as e:
        print(f"Ошибка парсинга на итерации {i}: {e}")
        continue

    # 3️⃣ Расчёт метрик
    alloc_vec = np.array([allocation.get(p, 0) for p in players])
    cap_vec = np.array([caps[p] for p in players])
    contrib_vec = np.array([contribs[p] for p in players])
    final_caps = cap_vec + alloc_vec

    metrics = {
        "iteration": i + 1,
        "allocation_A": alloc_vec[0],
        "allocation_B": alloc_vec[1],
        "allocation_C": alloc_vec[2],
        "final_A": final_caps[0],
        "final_B": final_caps[1],
        "final_C": final_caps[2],
        "Gini": gini(final_caps),
        "Atkinson_0.5": atkinson(final_caps, 0.5),
        "Atkinson_1": atkinson(final_caps, 1.0),
        "Theil": theil(final_caps),
        "Deviation_from_proportional": rmse(alloc_vec, proportional_ideal(contrib_vec, total_budget)),
        "Min_final_cap": np.min(final_caps),
        "Principle": principle,
        "Justification": justification,
        "Coherence_check": coherence
    }

    results.append(metrics)

    # Пауза между запросами, чтобы не спамить API
    sleep(1)

# 4️⃣ Формируем DataFrame
df_results = pd.DataFrame(results)
display(df_results)

{
  "allocation": { "A": 30, "B": 40, "C": 30 },
  "principle": "needs-based",
  "justification": "Бот A нуждается в поддержке, чтобы улучшить свою ситуацию, поэтому получает больше. Бот B, несмотря на большой вклад, уже имеет высокий капитал, поэтому получает меньше. Бот C также сохраняет значительное богатство и получает соответствующую долю.",
  "coherence_check": "Распределение учитывает нужды участников и их текущие капиталы, что делает решение логичным и последовательным."
}
{
  "allocation": { "A": 50, "B": 30, "C": 20 },
  "principle": "needs-based",
  "justification": "Бот A нуждается в поддержке, так как имеет наименьший капитал и вклад. Бот B также получает значительное количество, так как его вклад высок, но не настолько, чтобы преобладать над нуждами A. Бот C, имея наибольший капитал, получает наименьшую долю.",
  "coherence_check": "Распределение соответствует принципу нужд, так как Бот A получает больше для удовлетворения своих потребностей, несмотря на вклад других учас

,iteration,allocation_A,allocation_B,allocation_C,final_A,final_B,final_C,Gini,Atkinson_0.5,Atkinson_1,Theil,Deviation_from_proportional,Min_final_cap,Principle,Justification,Coherence_check
0,1,30,40,30,35,90,130,0.248366,0.060797,0.126360,0.114994,28.284271,35,needs-based,"Бот A нуждается в поддержке, чтобы улучшить св...",Распределение учитывает нужды участников и их ...
1,2,50,30,20,55,80,120,0.169935,0.024699,0.049120,0.049366,37.416574,55,needs-based,"Бот A нуждается в поддержке, так как имеет наи...","Распределение соответствует принципу нужд, так..."
2,3,40,30,30,45,80,130,0.222222,0.043227,0.086596,0.085354,35.590261,45,needs-based,"Бот A нуждается в большем количестве ресурсов,...",Распределение соответствует принципу учета нуж...
3,4,30,40,30,35,90,130,0.248366,0.060797,0.126360,0.114994,28.284271,35,needs-based,"Бот A наиболее нуждается в ресурсе, так как им...","Распределение отражает принципы нужды, где наи..."
4,5,30,40,30,35,90,130,0.248366,0.060797,0.126360,0.114994,28.284271,35,needs-based,"Бот A нуждается в большем количестве ресурсов,...","Распределение соответствует принципу нужды, по..."
5,6,40,30,30,45,80,130,0.222222,0.043227,0.086596,0.085354,35.590261,45,needs-based,"Бот A наиболее нуждается в ресурсах, так как и...",Распределение учитывает нужды участников и обе...
6,7,30,50,20,35,100,120,0.222222,0.056176,0.118956,0.104224,21.602469,35,needs-based,"Боту A выделено больше монет, так как он нужда...",Распределение основано на потребностях участни...
7,8,30,50,20,35,100,120,0.222222,0.056176,0.118956,0.104224,21.602469,35,needs-based,Монеты распределены с учетом потребностей учас...,"Распределение соответствует принципу нужды, та..."
8,9,30,40,30,35,90,130,0.248366,0.060797,0.126360,0.114994,28.284271,35,needs-based,Распределение основывается на потребностях уча...,Распределение соответствует принципу потребнос...
9,10,30,40,30,35,90,130,0.248366,0.060797,0.126360,0.114994,28.284271,35,needs-based,"Бот A является нуждающимся, поэтому получил на...",Распределение учитывает нужды участников и их ...


In [15]:
df_results.to_excel("test_experiment.xlsx",index=False)